In [1]:
import os

%load_ext dotenv
%dotenv

FEATURE_COLUMNS = ['player_rating_home_player_1', 'player_rating_home_player_2', 'player_rating_home_player_3',
                   'player_rating_home_player_4', 'player_rating_home_player_5',
                   'player_rating_home_player_6', 'player_rating_home_player_7', 'player_rating_home_player_8',
                   'player_rating_home_player_9', 'player_rating_home_player_10',
                   'player_rating_home_player_11', 'player_rating_away_player_1', 'player_rating_away_player_2',
                   'player_rating_away_player_3', 'player_rating_away_player_4',
                   'player_rating_away_player_5', 'player_rating_away_player_6', 'player_rating_away_player_7',
                   'player_rating_away_player_8', 'player_rating_away_player_9',
                   'player_rating_away_player_10', 'player_rating_away_player_11', 'ewm_home_team_goals',
                   'ewm_away_team_goals', 'ewm_home_team_goals_conceded', 'ewm_away_team_goals_conceded',
                   'points_home', 'points_away', 'home_weighted_wins', 'away_weighted_wins', 'avg_home_team_rating',
                   'avg_away_team_rating', 'home_streak_wins', 'away_streak_wins', 'ewm_shoton_home',
                   'ewm_shoton_away', 'ewm_possession_home', 'ewm_possession_away', 'avg_home_rating_attack',
                   'avg_away_rating_attack', 'avg_away_rating_defence', 'avg_home_rating_defence',
                   'average_rating_home', 'average_rating_away', 'num_top_players_home', 'num_top_players_away',
                   'ewm_home_team_goals_conceded_x_ewm_shoton_home', 'attacking_strength_home',
                   'attacking_strength_away', 'attacking_strength_diff', 'result_match']

In [2]:
import boto3
from sagemaker.session import Session
from sagemaker import clarify

default_prefix = "sagemaker/DEMO-sagemaker-clarify"
region = os.environ.get("AWS_REGION")
default_bucket = os.environ["BUCKET"]
execution_role = os.environ["ROLE"]
sagemaker_session = Session()

clarify_processor = clarify.SageMakerClarifyProcessor(
    role=execution_role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    sagemaker_session=sagemaker_session
)

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\kamil\AppData\Local\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\kamil\AppData\Local\sagemaker\sagemaker\config.yaml


In [3]:
from io import StringIO
import pandas as pd

s3 = boto3.resource('s3')

train_path = os.getenv("TRAIN_PATH")
obj = s3.Object(default_bucket, train_path)
data = obj.get()['Body']

In [4]:
train_df = pd.read_csv(data, header=None)
train_df.columns = FEATURE_COLUMNS

mapping = {1.0: 'home_win', 0.0: 'home_not_win'}
train_df['result_match'] = train_df['result_match'].map(mapping)
train_df.head()

,player_rating_home_player_1,player_rating_home_player_2,player_rating_home_player_3,player_rating_home_player_4,player_rating_home_player_5,player_rating_home_player_6,player_rating_home_player_7,player_rating_home_player_8,player_rating_home_player_9,player_rating_home_player_10,...,avg_home_rating_defence,average_rating_home,average_rating_away,num_top_players_home,num_top_players_away,ewm_home_team_goals_conceded_x_ewm_shoton_home,attacking_strength_home,attacking_strength_away,attacking_strength_diff,result_match
0,70.0,70.0,70.0,70.0,70.0,70.0,73.0,69.0,74.0,75.0,...,70.0,71.000000,71.545455,0.0,0.0,7.92254,47.886667,55.174242,-7.287576,home_not_win
1,73.0,73.0,73.0,73.0,73.0,73.0,75.0,73.0,74.0,75.0,...,73.0,73.363636,82.181818,0.0,3.0,9.77076,31.216102,41.378109,-10.162008,home_not_win
2,74.0,74.0,74.0,74.0,74.0,74.0,74.0,72.0,73.0,74.0,...,74.0,74.000000,84.818182,0.0,10.0,0.00000,38.541667,27.056962,11.484705,home_not_win
3,86.0,86.0,86.0,86.0,86.0,86.0,85.0,86.0,81.0,87.0,...,86.0,85.363636,74.363636,10.0,0.0,5.19080,16.898406,38.219895,-21.321489,home_win
4,68.0,68.0,68.0,68.0,68.0,68.0,75.0,73.0,77.0,73.0,...,68.0,70.818182,76.727273,0.0,0.0,0.21420,43.814371,34.459459,9.354912,home_not_win


In [12]:
train_df['num_top_players_home'].describe()

count    4255.000000
mean        0.874971
std         2.204277
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max        11.000000
Name: num_top_players_home, dtype: float64

In [5]:
from sagemaker.s3 import S3Uploader

S3_LOCATION = f"s3://{default_bucket}/football"

df_local_path = "train_df_clarify.csv"
train_df.to_csv(df_local_path, index=False)

s3_data_input_path = S3Uploader.upload(local_path=df_local_path, desired_s3_uri=f"{S3_LOCATION}/clarify", sagemaker_session=sagemaker_session)

os.remove(df_local_path)

In [6]:
bias_report_output_path = "s3://{}/{}/clarify-bias".format(default_bucket, default_prefix)
bias_data_config = clarify.DataConfig(
    s3_data_input_path=s3_data_input_path,
    s3_output_path=bias_report_output_path,
    label="result_match",
    headers=train_df.columns.to_list(),
    dataset_type="text/csv",
)

In [7]:
MODEL_PACKAGE_GROUP = os.environ["MODEL_PACKAGE_GROUP"]

sm = boto3.client("sagemaker")
model_packages = sm.list_model_packages(ModelPackageGroupName=MODEL_PACKAGE_GROUP, SortBy="CreationTime", SortOrder="Descending")

model_package = [pk for pk in model_packages["ModelPackageSummaryList"] 
                 if pk["ModelApprovalStatus"] == "Approved"][0]

model_package_arn = model_package["ModelPackageArn"]

models = sm.search(
    Resource='Model',
    SearchExpression={
        'Filters': [
            {
                'Name': 'Model.Containers.ModelPackageName',
                'Operator': 'Equals',
                'Value': model_package_arn
            },
        ]
    }
)["Results"]

model_name = models[0]["Model"]["Model"]["ModelName"]
print(model_name)

football-endpoint-model-shadow-0616192632


In [8]:
content_template = '{"confidence":$features}'

model_config = clarify.ModelConfig(
    model_name=model_name,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    accept_type="application/jsonlines",
    content_type="text/csv",
)

In [9]:
predictions_config = clarify.ModelPredictedLabelConfig(
    label='prediction',
    probability='confidence',
    probability_threshold=0.65,
)

In [10]:
bias_config = clarify.BiasConfig(
    label_values_or_threshold=['home_win'], facet_name="num_top_players_home", facet_values_or_threshold=[0.5], group_name="home_weighted_wins"
)

In [11]:
from sagemaker.experiments import Run

with Run(
    experiment_name='tracking-bias-explainability',
    run_name="bias-only",
    sagemaker_session=sagemaker_session,
) as run:
    clarify_processor.run_bias(
        data_config=bias_data_config,
        bias_config=bias_config,
        model_config=model_config,
        model_predicted_label_config=predictions_config,
        pre_training_methods="all",
        post_training_methods="all",
    )

INFO:sagemaker.experiments.run:The run (bias-only) under experiment (tracking-bias-explainability) already exists. Loading it.
INFO:sagemaker.clarify:Analysis Config: {'dataset_type': 'text/csv', 'headers': ['player_rating_home_player_1', 'player_rating_home_player_2', 'player_rating_home_player_3', 'player_rating_home_player_4', 'player_rating_home_player_5', 'player_rating_home_player_6', 'player_rating_home_player_7', 'player_rating_home_player_8', 'player_rating_home_player_9', 'player_rating_home_player_10', 'player_rating_home_player_11', 'player_rating_away_player_1', 'player_rating_away_player_2', 'player_rating_away_player_3', 'player_rating_away_player_4', 'player_rating_away_player_5', 'player_rating_away_player_6', 'player_rating_away_player_7', 'player_rating_away_player_8', 'player_rating_away_player_9', 'player_rating_away_player_10', 'player_rating_away_player_11', 'ewm_home_team_goals', 'ewm_away_team_goals', 'ewm_home_team_goals_conceded', 'ewm_away_team_goals_concede

INFO:sagemaker-clarify-processing:Starting SageMaker Clarify Processing job
INFO:analyzer.data_loading.data_loader_util:Analysis config path: /opt/ml/processing/input/config/analysis_config.json
INFO:analyzer.data_loading.data_loader_util:Analysis result path: /opt/ml/processing/output
INFO:analyzer.data_loading.data_loader_util:This host is algo-1.
INFO:analyzer.data_loading.data_loader_util:This host is the leader.
INFO:analyzer.data_loading.data_loader_util:Number of hosts in the cluster is 1.
INFO:sagemaker-clarify-processing:Running Python / Pandas based analyzer.
INFO:analyzer.data_loading.data_loader_factory:Dataset type: text/csv uri: /opt/ml/processing/input/data
/usr/local/lib/python3.9/site-packages/analyzer/data_loading/data_readers/csv_data_reader.py:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_tmp, ignore_index=True)
/usr/local/lib/python3.9/site-packages/analyz